# Creating a Model 

In [1]:
import importlib, os, utils, urllib
from datetime import datetime, timedelta
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import pandas as pd
import numpy as np

2023-05-10 10:13:43.346426: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 10:13:43.400255: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 10:13:43.401548: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 10:13:44.344646: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# importlib.reload(utils)
# all_zones_df = utils.clean_raw_webscraper_data('output_data/All_Zones_2021-22_Season_reports_data.csv')
# all_zones_df = utils.add_weather_to_reports(all_zones_df)
# all_zones_df.to_csv('output_data/All_Zones_2021-22_Season_weather_and_reports_data.csv', index=False)

In [16]:
# all_zones_df = pd.read_csv('output_data/all_zones_all_data.csv')
all_zones_df = pd.read_csv('input_data/Stevens_Pass_Current_Season_reports_data.csv')

data = all_zones_df.drop(columns=['bottom_line_text', 'problem_type_text', 'forecast_discussion_text'])

data['combined_text'] = data['combined_text'].astype(str)
data['combined_text'] = data['combined_text'].apply(lambda x: word_tokenize(x.lower()))
data.rename(columns={'combined_text': 'tokens'}, inplace=True)

In [17]:

data

,date,zone,overall_risk,above_treeline_risk,near_treeline_risk,below_treeline_risk,tokens,tavg,tmin,tmax,prcp,wdir,pres,tsun
0,2023-04-15,stevens pass,1.0,1.0,1.0,1.0,[nan],7.6,3.1,11.1,1.3,165.1,1018.9,0.0
1,2023-04-14,stevens pass,1.0,1.0,1.0,1.0,"[warming, temperature, sun, maintain, slight, ...",6.8,0.3,13.0,0.0,254.3,1016.4,0.0
2,2023-04-13,stevens pass,2.0,2.0,2.0,1.0,"[warming, temperature, sun, maintain, threat, ...",5.9,1.4,10.5,1.1,244.6,1015.9,0.0
3,2023-04-12,stevens pass,2.0,2.0,2.0,1.0,"[mixed, cloud, sun, maintain, chance, triggeri...",5.4,0.5,10.7,1.1,251.1,1017.0,0.0
4,2023-04-11,stevens pass,2.0,2.0,2.0,2.0,"[clearing, sky, bring, chance, triggering, wet...",5.8,2.1,9.7,5.8,223.2,1015.3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,2022-11-28,stevens pass,2.0,2.0,2.0,1.0,"[trigger, avalanche, recent, snow, fell, weeke...",0.1,-3.7,2.5,2.9,240.2,1011.9,0.0
141,2022-11-28,stevens pass,3.0,3.0,2.0,1.0,"[forecast, updated, reflect, higher, expected,...",0.1,-3.7,2.5,2.9,240.2,1011.9,0.0
142,2022-11-27,stevens pass,2.0,2.0,2.0,1.0,"[rapid, doubling, snowpack, accompanied, wind,...",3.4,0.2,6.4,7.1,228.4,1013.0,0.0
143,2022-11-26,stevens pass,3.0,3.0,3.0,2.0,"[avalanche, danger, increase, storm, deposit, ...",2.8,-0.9,5.9,2.0,189.4,1026.6,0.0


In [54]:
# Training the word to vec model:
model = Word2Vec(sentences=data['tokens'], vector_size=100, window=5, min_count=1, workers=4)
model.save('models/first_word2vec.model')

model = Word2Vec.load('models/first_word2vec.model')
# Create word embeddings lookup dictionary
word_embeddings = {}

for word in model.wv.index_to_key:
    word_embeddings[word] = model.wv[word]

word = 'cornice'
model.wv.most_similar(word)

[('area', 0.9997828602790833),
 ('like', 0.9997230172157288),
 ('day', 0.9997174739837646),
 ('travel', 0.9997053742408752),
 ('especially', 0.9997028112411499),
 ('one', 0.9996917247772217),
 ('snow', 0.999691367149353),
 ('tree', 0.9996909499168396),
 ('find', 0.9996904730796814),
 ('snowpack', 0.9996773600578308)]

In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from bokeh.plotting import figure, show, output_file

# Load the dataframe
# df = pd.read_csv('your_dataframe.csv')
df = data

# Select the features you want to use for training
features = ['tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'pres', 'tsun']

# Preprocess the data
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])  # Normalize the numerical features

# Convert text tokens to numerical sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['tokens'])

sequences = tokenizer.texts_to_sequences(df['tokens'])
max_seq_length = max(len(seq) for seq in sequences)
sequences = pad_sequences(sequences, maxlen=max_seq_length)

tokenized_text = [str(tokens).split() for tokens in df['tokens']]
# Train Word2Vec embeddings
model = Word2Vec(sentences=tokenized_text, vector_size=50, window=7, min_count=7, workers=10)

# Save the trained model
model.save('models/first_word2vec.model')

# Load pre-trained word embeddings (Word2Vec or GloVe)
word_embeddings = Word2Vec.load('models/first_word2vec.model').wv

# Create an embedding matrix
embedding_dim = word_embeddings.vector_size
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in word_embeddings.key_to_index:
        embedding_matrix[i] = word_embeddings[word]

# Prepare the input and output data
X = sequences
y = df['overall_risk'].values

# Define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_seq_length, trainable=False))
model.add(LSTM(16))
model.add(Dense(1, activation='linear'))

# Compile and train the model
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=8)


Epoch 1/20


2023-05-10 11:29:36.329952: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-10 11:29:36.334465: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-10 11:29:36.337415: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

19/19 [==============================] - 4s 96ms/step - loss: 5.4048
Epoch 2/20
19/19 [==============================] - 2s 92ms/step - loss: 4.0670
Epoch 3/20
19/19 [==============================] - 2s 91ms/step - loss: 0.8193
Epoch 4/20
19/19 [==============================] - 2s 91ms/step - loss: 0.5038
Epoch 5/20
19/19 [==============================] - 2s 94ms/step - loss: 0.4901
Epoch 6/20
19/19 [==============================] - 2s 93ms/step - loss: 0.4909
Epoch 7/20
19/19 [==============================] - 2s 99ms/step - loss: 0.4899
Epoch 8/20
19/19 [==============================] - 2s 91ms/step - loss: 0.4979
Epoch 9/20
19/19 [==============================] - 2s 96ms/step - loss: 0.4952
Epoch 10/20
19/19 [==============================] - 2s 97ms/step - loss: 0.4929
Epoch 11/20
19/19 [==============================] - 2s 99ms/step - loss: 0.4932
Epoch 12/20
19/19 [==============================] - 2s 96ms/step - loss: 0.5065
Epoch 13/20
19/19 [=============================

In [30]:

# Predict for the next 7 days
last_sequence = sequences[-1]  # Get the last sequence from the input data

predictions = []
for _ in range(7):
    # next_sequence = np.concatenate([last_sequence[1:], np.zeros(1)])  # Shift the sequence by one position
    next_sequence = next_sequence.reshape(1, -1)  # Reshape to match the input shape
    prediction = model.predict(next_sequence)[0][0]  # Make the prediction
    predictions.append(prediction)

    # Update the last sequence with the predicted value
    last_sequence[-1] = prediction
    last_sequence = last_sequence.reshape(1, -1)

# Print the predictions for the next 7 days
print("Predictions for the next 7 days:")
for i, prediction in enumerate(predictions, 1):
    print(f"Day {i}: {prediction}")

# Chart the predictions
days = range(1, 8)
risk_levels = predictions

p = figure(title="Risk Level Predictions for Next 7 Days", x_axis_label="Day", y_axis_label="Risk")


1/1 [==============================] - 0s 40ms/step
Predictions for the next 7 days:
Day 1: 2.3292155265808105
Day 2: 2.3292155265808105
Day 3: 2.3292155265808105
Day 4: 2.3292155265808105
Day 5: 2.3292155265808105
Day 6: 2.3292155265808105
Day 7: 2.3292155265808105


In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from bokeh.plotting import figure, show, output_file

# Load the dataframe
# df = pd.read_csv('your_dataframe.csv')
df = data

# Select the features you want to use for training
features = ['tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'pres', 'tsun']

# Preprocess the data
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])  # Normalize the numerical features

# Convert text tokens to numerical sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['tokens'])

sequences = tokenizer.texts_to_sequences(df['tokens'])
max_seq_length = max(len(seq) for seq in sequences)
sequences = pad_sequences(sequences, maxlen=max_seq_length)

tokenized_text = [str(tokens).split() for tokens in df['tokens']]
# Train Word2Vec embeddings
model = Word2Vec(sentences=tokenized_text, vector_size=50, window=7, min_count=7, workers=10)

# Save the trained model
model.save('models/first_word2vec.model')

# Load pre-trained word embeddings (Word2Vec or GloVe)
word_embeddings = Word2Vec.load('models/first_word2vec.model').wv

# Create an embedding matrix
embedding_dim = word_embeddings.vector_size
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in word_embeddings.key_to_index:
        embedding_matrix[i] = word_embeddings[word]

# Prepare the input and output data
X = sequences
y = df['overall_risk'].values

# Split the data into training and testing sets
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_seq_length, trainable=False))
model.add(LSTM(16))
model.add(Dense(1, activation='linear'))

# Compile and train the model
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(X_train, y_train, epochs=20, batch_size=8)

# Predict on the second half of the training data
X_train_pred = model.predict(X_train[train_size:])
y_train_pred = X_train_pred.flatten()

# Plot the predictions and actual results
output_file("predictions_plot.html")
p = figure(title="Model Predictions vs Actual Results", x_axis_label="Data Index", y_axis_label="Overall Risk")
p.line(range(train_size, len(y_train)), y_train_pred, legend_label="Predictions", line_color="red")
p.line(range(train_size, len(y_train)), y_train, legend_label="Actual", line_color="blue")
show(p)

SyntaxError: incomplete input (820452495.py, line 83)

In [44]:

# Plot the training loss
output_file("loss_plot.html")
p1 = figure(title="Training Loss", x_axis_label="Epoch", y_axis_label="Loss")
p1.line(range(1, len(history.history['loss']) + 1), history.history['loss'])
# Saving the plot to the output_file:
show(p1)

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/jaymin/snap/code/common/.cache/gio-modules/libgiolibproxy.so


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/jaymin/snap/code/common/.cache/gio-modules/libgiolibproxy.so
